<a href="https://colab.research.google.com/github/harjeet88/LLM_experiemnts/blob/main/RAG/LLM_RAG_sql_lite_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sqlite3
import torch
import numpy as np #Import numpy
from transformers import AutoTokenizer, AutoModel, pipeline

In [2]:
# Define the model and tokenizer
#model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_name = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
#model_name = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [3]:
# Setting up SQLite connection
conn = sqlite3.connect('rag_example.db')
cur = conn.cursor()

In [4]:
# Create table with pgvector extension
cur.execute("""
CREATE TABLE IF NOT EXISTS documents (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    content TEXT,
    embedding BLOB
);
""")
conn.commit()

In [5]:
# Function to compute embeddings
def compute_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.pooler_output[0].numpy()

In [6]:
# Inserting documents into the database
documents = [
    "The quick brown fox jumps over the lazy dog.",
    "The rain in Spain stays mainly in the plain.",
    "In a hole in the ground there lived a hobbit."
]

In [7]:
for doc in documents:
    embedding = compute_embedding(doc).tobytes()  # Convert numpy array to bytes
    cur.execute("INSERT INTO documents (content, embedding) VALUES (?, ?)", (doc, embedding))
conn.commit()

In [15]:
# Function to retrieve documents based on query
def retrieve_documents(query, top_k=3):
    query_embedding = compute_embedding(query)
    cur.execute("SELECT content, embedding FROM documents")
    rows = cur.fetchall()

    # Compute cosine similarity between query embedding and document embeddings
    similarities = []
    for content, embedding in rows:
        doc_embedding = torch.tensor(list(embedding), dtype=torch.float32).view(1, -1)
        similarity = torch.nn.functional.cosine_similarity(torch.tensor(query_embedding), doc_embedding)
        similarities.append((content, similarity.item()))

    # Sort by similarity and return top_k results
    similarities.sort(key=lambda x: x[1], reverse=True)
    return [content for content, _ in similarities[:top_k]]

In [9]:
# Function to retrieve documents based on query
def retrieve_documents(query, top_k=3):
    query_embedding = compute_embedding(query)
    cur.execute("SELECT content, embedding FROM documents")
    rows = cur.fetchall()

    # Compute cosine similarity between query embedding and document embeddings
    similarities = []
    for content, embedding in rows:
        # Convert the byte embedding back to a numpy array, then a tensor
        #doc_embedding = np.frombuffer(embedding, dtype=np.float32)
        #doc_embedding = torch.tensor(doc_embedding).view(1, -1) #View is added to ensure correct dimensions
        doc_embedding = np.frombuffer(embedding, dtype=np.float32)
        doc_embedding = doc_embedding.reshape(1, -1) # Reshape the array to be 1x768
        doc_embedding = torch.tensor(doc_embedding)
        similarity = torch.nn.functional.cosine_similarity(torch.tensor(query_embedding), doc_embedding)
        similarities.append((content, similarity.item()))

    # Sort by similarity and return top_k results
    similarities.sort(key=lambda x: x[1], reverse=True)
    return [content for content, _ in similarities[:top_k]]

In [12]:
# Function to retrieve documents based on query
def retrieve_documents(query, top_k=3):
    query_embedding = compute_embedding(query)
    cur.execute("SELECT content, embedding FROM documents")
    rows = cur.fetchall()

    # Compute cosine similarity between query embedding and document embeddings
    similarities = []
    for content, embedding in rows:
        # Convert the byte embedding back to a numpy array, then a tensor
        doc_embedding = np.frombuffer(embedding, dtype=np.float32)
        doc_embedding = doc_embedding.reshape(1, -1) # Reshape the array to be 1x768
        doc_embedding = torch.tensor(doc_embedding)
        similarity = torch.nn.functional.cosine_similarity(torch.tensor(query_embedding), doc_embedding)
        similarities.append((content, similarity.item()))

    # Sort by similarity and return top_k results
    similarities.sort(key=lambda x: x[1], reverse=True)
    return [content for content, _ in similarities[:top_k]]

In [16]:
# Function to generate a response
def generate_response(query):
    retrieved_docs = retrieve_documents(query)
    combined_text = " ".join(retrieved_docs)
    generator = pipeline("text-generation", model="gpt2")
    response = generator(f"Query: {query}\nDocuments: {combined_text}\nAnswer:", max_length=100)
    return response[0]['generated_text']

In [17]:
# Example query
query = "Tell me about foxes and rain."
response = generate_response(query)
print(response)

RuntimeError: The size of tensor a (768) must match the size of tensor b (1536) at non-singleton dimension 1

In [11]:

# Close the database connection
cur.close()
conn.close()